En la presente etapa del estudio se busca determinar los hiperparámetros del modelo. Una vez que se tenga aquella combinación con menor pérdida, se pasa a estimar el modelo en una segunda etapa.

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/D2FM')
module_path = '/content/drive/My Drive/D2FM'
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
!pip install pykalman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=fa30d4abb1df13ca5ab3482f1d7111bc0797e33ed3277e6b3adc469b756b705d
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully built pykalman


In [5]:
from models.ddfm import *

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from sklearn.model_selection import ParameterGrid
from typing import Tuple
import numpy as np
from pykalman import KalmanFilter

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [7]:
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.metrics import mean_squared_error as mse
from typing import Tuple
from sklearn.preprocessing import StandardScaler

In [8]:
df_train = pd.read_csv('/content/drive/MyDrive/D2FM/scaled_train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/D2FM/scaled_test.csv')


In [13]:
df_train_idx=df_train.set_index("Index")
df_test_idx=df_test.set_index("Index")

# K=1 para determinar los hiperparámetros del modelo. Son 64 combinaciones en una sola validación. Se determina el "mejor" conjunto de hiperparámetros en base a la pérdida menor.

In [24]:
import time

start_time_total = time.time()  # Record the start time for the entire process

# Your existing code
r = 6
structure_encoders = [(r * 6, r * 4, r * 2, r),
                      (r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r), (r * 48, r * 24, r * 16, r)]

lags_inputs = list(range(2))
link_options = ['tanh', 'relu']  # New link options
batch_size_options = [24, 100]   # New batch size options

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Iterate over each link option
            for link_option in link_options:
                # Iterate over each batch size option
                for batch_size_option in batch_size_options:
                    # Initialize the DDFM with the current configuration and use_bias setting
                    ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                                optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link=link_option,
                                batch_size=batch_size_option, epochs=100, max_iter=1000)

                    # Start the timer for each individual model
                    start_time_model = time.time()

                    # Fit the model
                    ddfm.fit()

                    # Stop the timer for each individual model
                    end_time_model = time.time()

                    # Calculate the execution time for each individual model
                    execution_time_model = end_time_model - start_time_model

                    # Store the model, its configuration, final loss, and execution time
                    final_loss = ddfm.loss_now
                    model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss,
                                       execution_time_model, link_option, batch_size_option))

# Calculate the total execution time
end_time_total = time.time()
total_execution_time = end_time_total - start_time_total

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model, link_option, batch_size_option in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option} - Final Loss: {final_loss}")
    print(f"Execution Time for this model: {execution_time_model:.2f} seconds")

print(f"Total Execution Time for all models: {total_execution_time:.2f} seconds")


@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6062719455379042 - delta: 0.0001527789508580558 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6042899097289977 - delta: 0.0003065221216681004 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.6023843152966347 - delta: 3.685885736727355e-05 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.598491798566092 - delta: 0.00015911664960780816 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6002428834341017 - delta: 0.00022240091485573735 < 0.0005
@Info - Note: Sorti

In [25]:
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time, link_option, batch_size_option in model_info:
    # Update min_loss and min_loss_config if current loss is smaller
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option = min_loss_config
    print(f"Minimum Loss: {min_loss} with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}")
else:
    print("No models found or no valid loss values.")


Minimum Loss: 0.58967100937883 with encoder (288, 144, 96, 6), lag 1, use_bias True, link tanh, batch_size 100


Para menos combinaciones de hiperparámetros, queda como ejemplo:

In [21]:
import time

start_time_total = time.time()  # Record the start time for the entire process

# Your existing code
r = 6
structure_encoders = [(r * 6, r * 4, r * 2, r),
                      (r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r), (r * 48, r * 24, r * 16, r),]

lags_inputs = list(range(2))

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Initialize the DDFM with the current configuration and use_bias setting
            ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                        optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link='tanh',
                        epochs=100, max_iter=1000)

            # Start the timer for each individual model
            start_time_model = time.time()

            # Fit the model
            ddfm.fit()

            # Stop the timer for each individual model
            end_time_model = time.time()

            # Calculate the execution time for each individual model
            execution_time_model = end_time_model - start_time_model

            # Store the model, its configuration, final loss, and execution time
            final_loss = ddfm.loss_now
            model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss, execution_time_model))

# Calculate the total execution time
end_time_total = time.time()
total_execution_time = end_time_total - start_time_total

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting} - Final Loss: {final_loss}")
    print(f"Execution Time for this model: {execution_time_model:.2f} seconds")

print(f"Total Execution Time for all models: {total_execution_time:.2f} seconds")


@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.6000213739457754 - delta: 2.948570757007814e-05 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.5981605811324008 - delta: 0.00041992750411991146 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6002641262117545 - delta: 0.0003938048813976006 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5901988115578353 - delta: 0.00020263298102667096 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5949236383805527 - delta: 3.758155916742125e-05 < 0.0005
@Info - Note: Sort

In [23]:
ddfm6 = DDFM(df_train_idx, structure_encoder=(288, 144, 96, 6), lags_input=1, factor_oder=1,
                             use_bias=False, link='tanh', max_iter=1000)
ddfm6.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.589643687159019 - delta: 0.00012052289014438879 < 0.0005
